<a href="https://colab.research.google.com/github/gianluigilopardo/Open-World-Recognition/blob/main/our_modifications/incremental_learning/ICARL/knn/iCaRLMain_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RUN KNN WITH BCE BCE

# Import GitHub repository

In [1]:
import os
import logging
import sys

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
if not os.path.isdir('./owr'):
  !git clone https://github.com/gianluigilopardo/Open-World-Recognition.git
  !mv 'Open-World-Recognition' 'owr'

Cloning into 'Open-World-Recognition'...
remote: Enumerating objects: 480, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 480 (delta 141), reused 146 (delta 107), pack-reused 257
Receiving objects: 100% (480/480), 2.07 MiB | 10.78 MiB/s, done.
Resolving deltas: 100% (284/284), done.


# Import packages

In [4]:
from owr.our_modifications.incremental_learning.ICARL.knn import ResNet
from owr.our_modifications.incremental_learning.ICARL.knn.dataset import Subset
from owr.our_modifications.incremental_learning.ICARL.knn.icarl import classify
from owr.our_modifications.incremental_learning.ICARL.knn.icarl import incremental_train
from owr.our_modifications.incremental_learning.ICARL.knn.icarl import update_representation
from owr.our_modifications.incremental_learning.ICARL.knn.icarl import construct_exemplar_set
from owr.our_modifications.incremental_learning.ICARL.knn.icarl import reduce_exemplars
from owr.our_modifications.incremental_learning.ICARL.knn.icarl import generate_new_exemplars
from owr.our_modifications.incremental_learning.ICARL.knn.models import compute_loss
from owr.our_modifications.incremental_learning.ICARL.knn.models import train_network
from owr.our_modifications.incremental_learning.ICARL.knn import params
from owr.our_modifications.incremental_learning.ICARL.knn import utils
from owr.our_modifications.incremental_learning.ICARL.knn.utils import get_classes_names
from owr.our_modifications.incremental_learning.ICARL.knn.utils import get_task_indexes
from owr.our_modifications.incremental_learning.ICARL.knn.utils import splitter
from owr.our_modifications.incremental_learning.ICARL.knn.utils import map_splits
from owr.our_modifications.incremental_learning.ICARL.knn.utils import get_classes
from owr.our_modifications.incremental_learning.ICARL.knn.utils import get_indexes
import owr.our_modifications.incremental_learning.ICARL.knn.models

In [5]:
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random

In [6]:
pip install seaborn

In [7]:
print(params.SEED)
print(params.NUM_WORKERS)

42
2


# Define Datasets

In [8]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
from torchvision import datasets
trainDS = datasets.cifar.CIFAR100( 'data', train=True, download=True, transform=train_transformer)
testDS = datasets.cifar.CIFAR100( 'data', train=False, download=True, transform=test_transformer)


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [10]:
splits = splitter()

# Define Network

In [11]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [12]:
exemplars = [None]*params.NUM_CLASSES

test_indexes =  []
accs_train = []
accs_test = []
classifier = 'nme'

In [13]:
classifier = KNeighborsClassifier()
clf_params = {'n_neighbors' : np.arange(1,19,2)}

In [14]:
def tmp_classify(images, exemplars, model, task, train_dataset, mean=None, classifier='nme', clf_params=None):
    if classifier == 'nme':
        preds, mean = classify_nme(images, exemplars, model, task, train_dataset, mean)
        return preds, mean
    else:
        preds, mean = classify_models(images, exemplars, model, task, train_dataset, mean, classifier, clf_params)
        return preds, mean

def classify_models(images, exemplars, model, task, train_dataset, cv, classifier, clf_params):
    splits = utils.splitter()
    model.train(False)
    m = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        images = images.float().to(params.DEVICE)
        x_test = model(images)
        x_test = m(x_test)
    if cv == None:
        analyzed_classes = []
        for i in range(int(task / params.TASK_SIZE)+1):
            analyzed_classes = np.concatenate((analyzed_classes, splits[i]))
        l = []
        for k in range(len(analyzed_classes)):
            class_k = int(analyzed_classes[k])
            l.extend(exemplars[class_k])

        transformer = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        subset = Subset(train_dataset, l, transformer)
        data_loader = DataLoader(subset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE)
        df = pd.DataFrame(columns = ['data', 'labels'])
        for images, labels, idxs in data_loader:
            with torch.no_grad():
                images = images.float().to(params.DEVICE)
                x = model(images)
                x = m(x)
                tmp = pd.DataFrame()
                tmp['data'] = x.to('cpu')
                tmp['labels'] = labels.to('cpu')
                df = df.append(tmp)
        cv = GridSearchCV(classifier, clf_params)
        cv.fit(list(df['data']),list(df['labels']) )
        print(cv.best_params_)
    preds = cv.predict(x_test.to('cpu'))
    return torch.tensor(preds), cv

In [ ]:
for task in range(0, params.NUM_TASKS*params.TASK_SIZE, params.TASK_SIZE):
    train_indexes = get_task_indexes(trainDS, task)
    test_indexes = test_indexes + get_task_indexes(testDS, task)

    train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
    test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

    train_loader = DataLoader( train_dataset, num_workers=2, batch_size=params.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader( test_dataset, num_workers=2, batch_size=params.BATCH_SIZE , shuffle=True )
    print(task)
    ICaRL, exemplars = incremental_train(trainDS, ICaRL, exemplars, task, train_transformer, loss_version='opt1')
    col = []
    for i,x in enumerate( splits[ :int(task/10) + 1]) : 
        v = np.array(x)
        col = np.concatenate( (col,v), axis = None)
        col = col.astype(int)
    mean = None
    total = 0.0
    running_corrects = 0.0
    
    for img, lbl, _ in train_loader:
        img = img.float().to(params.DEVICE)
        preds, mean = tmp_classify(img, exemplars, ICaRL, task, trainDS, mean, classifier, clf_params)
        preds = preds.to(params.DEVICE)
        if classifier == 'nme':
            labels = map_splits(lbl, col).to(params.DEVICE)
        else:
            labels = lbl.to(params.DEVICE)
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'train accuracy = {accuracy}')
    accs_train.append(accuracy)

    total = 0.0
    running_corrects = 0.0
    tot_preds = []
    tot_lab = []
    for img, lbl, _ in test_loader:
        img = img.float().to(params.DEVICE)
        preds, _ = tmp_classify(img, exemplars, ICaRL, task, trainDS, mean, classifier, clf_params)
        preds = preds.to(params.DEVICE)
        if classifier == 'nme':
            labels = map_splits(lbl, col).to(params.DEVICE)
        else:
            labels = lbl.to(params.DEVICE)
        tot_preds = np.concatenate( ( tot_preds, preds.data.cpu().numpy() ) )
        tot_lab = np.concatenate( (tot_lab, labels.data.cpu().numpy()  ) )
        total += len(lbl)
        running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = float(running_corrects/total)
    print(f'task: {task}', f'test accuracy = {accuracy}')
    accs_test.append(accuracy)

0
Step: 0, Epoch: 0, Loss: 0.020933039486408234, Accuracy: 0.2102
Step: 0, Epoch: 1, Loss: 0.02003304660320282, Accuracy: 0.4148
Step: 0, Epoch: 2, Loss: 0.021279089152812958, Accuracy: 0.4764
Step: 0, Epoch: 3, Loss: 0.02608495019376278, Accuracy: 0.5402
Step: 0, Epoch: 4, Loss: 0.01541602611541748, Accuracy: 0.575
Step: 0, Epoch: 5, Loss: 0.033868249505758286, Accuracy: 0.6004
Step: 0, Epoch: 6, Loss: 0.02914546988904476, Accuracy: 0.6314


In [ ]:
torch.save(ICaRL, "./knnModel.pt")

In [ ]:
n = 1254
images_test = [n]
example_test_ds = Subset(testDS, images_test, transform = test_transformer)
example_test_loader = DataLoader( example_test_ds, num_workers=2, batch_size=params.BATCH_SIZE , shuffle=True )

In [ ]:
x = testDS.__getitem__(n)[0]

In [ ]:
import matplotlib.pyplot as plt
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
show(x)

In [ ]:
m = torch.nn.Softmax(dim=1)

In [ ]:
for img, lbl, _ in example_test_loader:
    img = img.float().to(params.DEVICE)
    classify_preds, _ = tmp_classify(img, exemplars, ICaRL, task, trainDS, mean, classifier, clf_params)
    outputs_preds = m(ICaRL(img, features = 0).to('cpu'))

In [ ]:
print(lbl, testDS.classes[lbl])

In [ ]:
df = pd.DataFrame(columns=['class_name', 'value'])

In [ ]:
for i, el in enumerate(outputs_preds.tolist()[0]):
  df.loc[i] = [testDS.classes[i], el]

In [ ]:
df.sort_values(by=['value'], ascending=False)

In [ ]:
np.argmax(outputs_preds.tolist())

In [ ]:
classify_preds